# Using Machine Learning to Predict Heterogeneous Treatment Effects

## Algorithms tailored for predicting outcomes can do poorly when predicting treatment effects

### Factors that strongly predict outcomes may not strongly predict treatment effects

$Y_i$: spending on a Lexus

$W_i$: seeing an online ad for a Lexus

$\ln Y_i=\beta_0+\beta_1 age_i +\beta_2 male_i + \beta_3 W_i+\beta_4 W_i \times male_i +\varepsilon_i$

How do outcomes vary by age? (A lot if $\beta_1$ is big)

How do treatment effects vary by age? (not at all!)

What do treatment effects vary by? (gender!)

In [ ]:
# Install econml
!pip install econml

In [ ]:
# import useful packages
import numpy as np
import pandas as pd
from sklearn.linear_model import LassoCV
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.model_selection import train_test_split
from econml.dml import CausalForestDML as CausalForest
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")

In [ ]:
# define parameters
n = 1000 # sample size
p = .5 # probability of seeing the ad
beta0=0
beta1=.2 # effect of age
beta2=-.025 # difference in average spending between males and females who don't see the ad ()
beta3=0 # effect of treatment among females
beta4=.05 # differential effect of treatment among males compared to females
sigeps=.02 # residual standard deviation of outcome

# generate some fake data
age=np.random.randint(low=18,high=61,size=(n,1))
male=np.random.randint(low=0,high=2,size=(n,1))
w=np.random.rand(n,1)>(1-p)
epsilon=sigeps*np.random.randn(n,1)
lny=beta0+beta1*age+beta2*male+beta3*w+beta4*w*male+epsilon

# assemble as dataframe
fakedata=pd.DataFrame(np.concatenate((lny,w,age,male),axis=1),columns=['lny','w','age','male'])
fakedata.feature_names=['age','male']

# fit trees
tree1 = DecisionTreeRegressor(max_depth=2)
tree1.fit(fakedata.loc[w==1,['age','male']],fakedata.loc[w==1,['lny']])
tree0 = DecisionTreeRegressor(max_depth=2)
tree0.fit(fakedata.loc[w==0,['age','male']],fakedata.loc[w==0,['lny']])

In [ ]:
# look at trees
print('Treated tree:')
d=tree.plot_tree(tree1,filled=True,feature_names=fakedata.feature_names)

In [ ]:
print('Untreated tree:')
d=tree.plot_tree(tree0,filled=True,feature_names=fakedata.feature_names)

Note that both trees used only age. As a result, the predicted treatment effects will not vary at all by gender, which in reality is the only feature upon which treatment effects depend.



### Non-Honest trees can exaggerate differences between groups

In [ ]:
#generate a bunch of predictors:
x1=np.random.randint(low=0,high=2,size=(n,1))
x2=np.random.randint(low=0,high=2,size=(n,1))
x3=np.random.randint(low=0,high=2,size=(n,1))
x4=np.random.randint(low=0,high=2,size=(n,1))
x5=np.random.randint(low=0,high=2,size=(n,1))
x6=np.random.randint(low=0,high=2,size=(n,1))
x7=np.random.randint(low=0,high=2,size=(n,1))
x8=np.random.randint(low=0,high=2,size=(n,1))
x9=np.random.randint(low=0,high=2,size=(n,1))
x10=np.random.randint(low=0,high=2,size=(n,1))

# Each of these predictors affects the outcome with coefficient = 1:
y=x1+x2+x3+x4+x5+x6+x7+x8+x9+x10+epsilon

# Now grow a tree:
predictors=np.concatenate((x1,x2,x3,x4,x5,x6,x7,x8,x9,x10),axis=1)
predictor_names=['x1','x2','x3','x4','x5','x6','x7','x8','x9','x10']
dishonest=DecisionTreeRegressor(max_depth=2)
dishonest.fit(predictors,y)
d=tree.plot_tree(dishonest,filled=True,feature_names=predictor_names)


According to our "model" for y, what should be the difference between outcomes in neighboring leaves from the same node? (one!) What are the observed differences in the tree? (greater than one!)

Solution: Honest trees, or double-sample trees, use a subset of the training set to grow the tree, and another subset to estimate means in each leaf

In [ ]:
train=np.random.randint(low=0,high=2,size=n)==1

# Grow tree on training set:
dt = DecisionTreeRegressor(max_depth=2)
dt.fit(predictors[train,:],y[train])
d=tree.plot_tree(dt,filled=True,feature_names=predictor_names)

In [ ]:
# But use estimation set to calculate means in each leaf:
leaves=dt.apply(predictors[~train])
yest=y[~train]
honest=dt
n_nodes = honest.tree_.node_count
for ii in range(n_nodes):
  if honest.tree_.children_left[ii]==-1: # means that the current node is a leaf
    estii=yest[leaves==ii]
    honest.tree_.value[ii]=estii.mean() # replace the leaf's value with the estimation set's mean

d=tree.plot_tree(honest,filled=True,feature_names=predictor_names)

The leaf means are not nearly as extreme as in the "dishonest" tree that uses the training observations to construct the predictions

## Random Causal Forest

In [ ]:
train, test = train_test_split(fakedata, test_size=0.2)
estimator = CausalForest(n_estimators=100)
estimator.fit(train['lny'],
              train['w'],
              X=train[fakedata.feature_names])
effects_train = estimator.effect(train[fakedata.feature_names])
effects_test = estimator.effect(test[fakedata.feature_names])
conf_intrvl = estimator.effect_interval(test[fakedata.feature_names])

In [ ]:
malefx_train=effects_train[train['male'].values==1]
maleage_train=train['age'].iloc[train['male'].values==1]
malefx_train.mean()




In [ ]:
femalefx_train=effects_train[train['male'].values==0]
femaleage_train=train['age'].iloc[train['male'].values==0]
femalefx_train.mean()

In [ ]:
malefx_test=effects_test[test['male'].values==1]
malefx_test.mean()

In [ ]:
femalefx_test=effects_test[test['male'].values==0]
femalefx_test.mean()

In [ ]:
fig = plt.figure()
ax = plt.axes()
ax.scatter(maleage_train,malefx_train,label='males');
ax.scatter(femaleage_train,femalefx_train,label='females');
ax.legend()
plt.title("Estimated Treatment effects")
plt.xlabel("age")
plt.ylabel("treatment effect");

Clearly doesn't nail the age profile (which should be flat) but does get the difference between men and women!

## Random Causal Forest: Predict the effects of job training


We are ready to apply machine learning to predict causal effects in a real-life setting: how do the effects of job training vary by an individual's characteristics? We will use data from the National Job Training Partnership study, a large-scale randomized evaluation of a publicly subsidized job training program for disadvantaged youth and young adults. Why would we care how the effects of a subsidized job training program vary by a person's characteristics?


We will use the JTPA evaluation dataset, which contains observations on about 14,000 individuals, some of whom were randomized to participate in job training ($z_i = 1$) and others who were not ($z_i = 0$).

To do on your own:

- load the dataset from the url `https://github.com/Mixtape-Sessions/Heterogeneous-Effects/raw/main/Labs/data/jtpahet.csv`
- define the outcome vector (call it `y`) to be the column labeled `foundjob`
- define the randomized assignment indicator (call it `z`) to be the column labeled `z`
- define the feature vector (call it `x`) to be all columns except `foundjob`, `z`, and `enroll`.


In [ ]:
# load the data

# define the variables

### Cheat


In [ ]:
data = pd.read_csv(
    "https://github.com/Mixtape-Sessions/Heterogeneous-Effects/raw/main/Labs/data/jtpahet.csv"
)
data

In [ ]:
y = data["foundjob"]
z = data["z"]
x = data.drop(["foundjob", "z", "enroll"], axis=1)
x

### Regression to get average effect


On your own: run a linear regression of the outcome on the random assignment indicator, `z`. Since this was a randomized experiment, we don't need controls!


### Cheat:


In [ ]:
import statsmodels.api as sm
rhs = sm.add_constant(
    data["z"]
)  # you have to add the constant yourself with statsmodels!
model = sm.OLS(data["foundjob"], rhs)
results = model.fit(cov_type="HC3")  # heteroskedasticity-robust
print(results.summary())

### Set up random forest


So far, so good? Now create a random causal forest object, and fit it with outcome `y`, treatment variable `z`, and feature matrix `x`.


In [ ]:
# On your own: create and fit random causal forest object

### Cheat


In [ ]:
rcf = CausalForest(n_estimators=500, discrete_treatment=True, criterion="het").fit(
    y, z, X=x
)

### Explore effects


Let's see what kind of heterogeneous effects our random causal forest predicted


In [ ]:
# calculate the predicted effects:
insamplefx = rcf.effect(x)
# plot a histogram of the estimated effects, with average effect overlaid
fig = plt.figure()
ax = plt.axes()
ax.hist(insamplefx, bins=30, density=True)
plt.axvline(rcf.ate_, color="k", linestyle="dashed", linewidth=1)
plt.suptitle("Estimated Treatment effects")
plt.title("ATE: {:.3g}".format(rcf.ate_[0]))
plt.show()

Let's visualize how these effects vary by prior earnings and education by making a heatmap


In [ ]:
import itertools

In [ ]:
# create a grid of values for education and prior earnings:
educgrid = np.arange(data["educ"].values.min(), data["educ"].values.max() + 1)
earngrid = np.arange(
    data["priorearn"].values.min(), data["priorearn"].values.max(), 5000
)
grid = pd.DataFrame(
    itertools.product(educgrid, earngrid), columns=["educ", "priorearn"]
)

We'll first visualize the effects among married, nonwhite females of average age:


In [ ]:
grid["age"] = data["age"].values.mean()  # set age to the average
grid["female"] = 1  # set female = 1
grid["nonwhite"] = 1  # set nonwhite = 1
grid["married"] = 1  # set married = 1
# need to re-order the columns to match x:
grid=grid[x.columns]
grid

To do on your own: calculate the predicted effects for each "observation" in the grid:


In [ ]:
# gridfx = # uncomment and fill in on your own!

### Cheat


In [ ]:
gridfx = rcf.effect(grid)

### Visualize effects with a heatmap:


In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig = plt.figure()
ax = plt.subplot()
main = ax.scatter(
    grid["educ"], grid["priorearn"], c=gridfx, cmap="plasma", marker="s", s=300
)
plt.suptitle("Estimated Treatment effects")
plt.title("Nonwhite married females")
plt.xlabel("years of education")
plt.ylabel("prior earnings")

# create an Axes on the right side of ax. The width of cax will be 5%
# of ax and the padding between cax and ax will be fixed at 0.05 inch.
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(main, cax=cax)
plt.show()

To do on your own: make similar visualizations for males, singles, whites, different ages, etc.
